In [2]:
import requests
import os

# Define the API endpoint and your access key
url = "https://api.pexels.com/v1/search"
access_key = "MgxBjp4CnSFtTvWe3SeHtkTGp5oUOBWVG5S0PYZAd37hV3T3h2yY4PHp"

# Define the headers with your access key
headers = {
    "Authorization": access_key
}

# Define the path to the images folder
path = "./images"

# Define the number of images to download
num_images = 80

# Define the query parameters
query_params = {
    "query": "random", 
    "per_page": num_images
}

# Create the images folder if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)

# Send the API request and download the images
response = requests.get(url, headers=headers, params=query_params)
json = response.json()
for i in range(num_images):
    image_url = json["photos"][i]["src"]["original"]
    image_id = json["photos"][i]["id"]
    image_extension = ".jpg" # change extension as per your requirement
    image_filename = f"{image_id}{image_extension}"
    image_path = os.path.join(path, image_filename)

    # Download the image and save it to the images folder
    response = requests.get(image_url)
    with open(image_path, "wb") as f:
        f.write(response.content)


In [1]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            try:
                return obj.decode('utf-8')
            except UnicodeDecodeError:
                return obj.decode('utf-8', 'replace')
        return json.JSONEncoder.default(self, obj)

# Path to the folder containing pictures
path_to_folder = './images'

# Create the metadata folder if it doesn't exist
if not os.path.exists('metadata'):
    os.mkdir('metadata')

# Loop over all pictures in the folder
for filename in os.listdir(path_to_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Open the image and get the EXIF data
        image = Image.open(os.path.join(path_to_folder, filename))
        exifdata = image.getexif()

        # Create a dictionary to store the metadata for this image
        metadata = {}

        # Loop over all EXIF tags and add them to the metadata dictionary
        for tag_id, value in exifdata.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata[tag] = value

        # Write the metadata dictionary to a JSON file in the metadata folder
        with open(os.path.join('metadata', filename + '.json'), 'w') as f:
            json.dump(metadata, f, cls=MyEncoder)

In [8]:
import os
import json
from colorthief import ColorThief
import cv2
from webcolors import *

# Path to the folder containing pictures
path_to_folder = './images'

# Dictionary to store metadata of all pictures
metadata = {}

# Loop over all pictures in the folder
for filename in os.listdir(path_to_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Open the image and get the dominant color
        image_path = os.path.join(path_to_folder, filename)
        color_thief = ColorThief(image_path)
        dominant_color = color_thief.get_color(quality=1)

        # Convert the RGB color to a name using webcolors
        color_name = ''
        try:
            color_name = hex_to_name(rgb_to_hex(dominant_color))
        except ValueError:
            pass
        
        # Add metadata to dictionary
        metadata[filename] = {
            "dominant_color": dominant_color,
            "color_name": color_name
        }

# Write the metadata dictionary to a JSON file
with open('metadata.json', 'w') as f:
    json.dump(metadata, f)

KeyboardInterrupt: 